<a href="https://colab.research.google.com/github/Sreekar-Kandhadai/pyspark-interview-questions/blob/main/Walmart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
We have two datasets:

Sessions Table: Contains records of when users started their sessions.
Order Summary Table: Contains records of orders placed by users along with their values.

We want to:
Find users who started a session and placed an order on the same day.
Calculate the total number of orders and the total order value for those users.

In [11]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *

spark=SparkSession.builder.appName("learning").getOrCreate()

sessions_data = [
    (1, 1, '2024-01-01 00:00:00'),
    (2, 2, '2024-01-02 00:00:00'),
    (3, 3, '2024-01-05 00:00:00'),
    (4, 3, '2024-01-05 00:00:00'),
    (5, 4, '2024-01-03 00:00:00'),
    (6, 4, '2024-01-03 00:00:00'),
    (7, 5, '2024-01-04 00:00:00'),
    (8, 5, '2024-01-04 00:00:00'),
    (9, 3, '2024-01-05 00:00:00'),
    (10, 5, '2024-01-04 00:00:00')
]


orders_data = [
    (1, 1, 152, '2024-01-01 00:00:00'),
    (2, 2, 485, '2024-01-02 00:00:00'),
    (3, 3, 398, '2024-01-05 00:00:00'),
    (4, 3, 320, '2024-01-05 00:00:00'),
    (5, 4, 156, '2024-01-03 00:00:00'),
    (6, 4, 121, '2024-01-03 00:00:00'),
    (7, 5, 238, '2024-01-04 00:00:00'),
    (8, 5, 70, '2024-01-04 00:00:00'),
    (9, 3, 152, '2024-01-05 00:00:00'),
    (10, 5, 171, '2024-01-04 00:00:00')
]


sessions_df = spark.createDataFrame(sessions_data, ["session_id", "user_id", "session_date"]).withColumn("session_date",to_date(col('session_date')))

sessions_df.show()
orders_df = spark.createDataFrame(orders_data, ["order_id", "user_id", "order_value", "order_date"]).withColumn("order_date",to_date(col('order_date')))
orders_df.show()

df1=sessions_df.alias('s').join(orders_df.alias('o'),(col('s.user_id') ==col('o.user_id')) & (col('s.session_date')==col('o.order_date')),"inner")

df1.show()

df2=df1.groupBy('s.user_id','s.session_date').agg(count('o.order_id').alias('total_orders'),sum('o.order_value').alias('total_order_value'))

df2.show()




+----------+-------+------------+
|session_id|user_id|session_date|
+----------+-------+------------+
|         1|      1|  2024-01-01|
|         2|      2|  2024-01-02|
|         3|      3|  2024-01-05|
|         4|      3|  2024-01-05|
|         5|      4|  2024-01-03|
|         6|      4|  2024-01-03|
|         7|      5|  2024-01-04|
|         8|      5|  2024-01-04|
|         9|      3|  2024-01-05|
|        10|      5|  2024-01-04|
+----------+-------+------------+

+--------+-------+-----------+----------+
|order_id|user_id|order_value|order_date|
+--------+-------+-----------+----------+
|       1|      1|        152|2024-01-01|
|       2|      2|        485|2024-01-02|
|       3|      3|        398|2024-01-05|
|       4|      3|        320|2024-01-05|
|       5|      4|        156|2024-01-03|
|       6|      4|        121|2024-01-03|
|       7|      5|        238|2024-01-04|
|       8|      5|         70|2024-01-04|
|       9|      3|        152|2024-01-05|
|      10|      5| 